In [1]:
import json
import codecs
import pandas as pd

In [2]:
IN_PATH = "samples/yachts-all-60k-90k.json"
OUT_PATH = 'samples/yachts.xlsx'

In [3]:
# проверяется по-словам. если не все слова но >0 - тип будет составлен из этих слов
TYPES = [
    'Jeanneau Sun Odyssey',
    'Jeanneau Sun Fast',
    
    'Bavaria Cruiser',
    'Bavaria Lagoon',

    'Beneteau Cyclades',
    'Beneteau Oceanis Clipper',
    
    'Dufour',
    'Fast',
    'Legend',
    'Elan Impression',
    'North Wind',
    'Cyclades',
    'Hunter',
    'Gib Sea',
    'Oceanis',
    'Seawolf',
    'Hanse',
    'Harmony',
]

def type_get_weight(t,name_orig):
    name = name_orig.lower()
    words = t.lower().split(" ")

    num_pass = 0
    for w in words:
        if not w in name:
            break

        num_pass = num_pass+1
    if num_pass == 0:
        return (0,"(other)")

    return (num_pass," ".join(words[:num_pass]).upper())

def parse_name(name_orig):
    weights = [type_get_weight(t,name_orig) for t in TYPES]
    sw = sorted(weights, key=lambda w:-1*w[0])
    
    return sw[0][1]
        
        
        

In [4]:
json_raw = codecs.open(IN_PATH, "r", "utf-8").read()
yw_raw = json.loads(json_raw)
print "yachts loaded: " + str(len(yw_raw))

# flatten specs
for y in yw_raw:
    fs = ""
    thruster = ""
    furling = ""
    
    for (k,v) in y['full_specs'].items():
        ln = k if len(v)==0 else k + ": " + v
        fs = fs + ln + "\n"
        
        lln = ln.lower()
        # special items
        if "thrust" in lln:
            thruster = thruster + ln + " "

        if "furling" in lln:
            furling = furling + ln + " "

    # parse type
    name_full = y['name'].split(u"\u00a0")[1]
    y['Type'] = parse_name(name_full)    
        
    # parse price to eur
    # "price": "EUR\u00a0\u00a090,000 (US$\u00a0100,467)\u00a0",

    tax = ""
    pr_eur = 0
    pr = y["price"].lower()
    if "tax paid" in pr:
        tax = "Paid"
    if "tax not paid" in pr:
        tax = "Not Paid"
    y['Tax'] = tax
    
    # Country
    country = y['located'].split(" ")[-1].strip()
    y["Country"] = country
    
    if pr.startswith("eur"):
        pr_eur = int(pr.split(u"\u00a0\u00a0")[1].split(" ")[0].replace(",",""))
        
    if pr.startswith(u"£"):
        pr_gpb = int(pr.split(u"\u00a0")[1].split(" ")[0].replace(",",""))
        pr_eur = pr_gpb * 1.16 # current rate GPB to EUR
        
    y['Price_KEUR'] = pr_eur/1000
    
    y["full_specs"] = fs
    y["Thruster"] = thruster
    y["Furling"] = furling
    y["FurlingMain"] = "main" in furling.lower()

yachts loaded: 986


In [5]:
yachts = pd.DataFrame.from_dict(yw_raw)

In [6]:
yachts

,Country,Furling,FurlingMain,Price_KEUR,Tax,Thruster,Type,full_specs,hull_material,id,loa,located,name,price,price_full,url,year
0,Malta,Furling mainsail Furling genoa,True,89.0000,Paid,,JEANNEAU SUN ODYSSEY,Teak sidedecks\nLOA: 13.30 m\nFurling mainsail...,Fiberglass,68071-2311144,44',Malta,1992 Jeanneau Sun Odyssey 44,"EUR 89,000 Tax Paid (US$ 99,351)","US$ 99,351",http://www.yachtworld.com/boats/1992/Jeanneau-...,1992
1,Germany,Furling genoa,False,62.0000,Not Paid,,DUFOUR,LOA: 13.40 m\nFresh Water Tanks: 2 (180 Liters...,Fiberglass,78634-2581151,44',"Greifswald, Germany",1998 Dufour 43,"EUR 62,185 Tax Not Paid (US$ 69,417)","US$ 69,417",http://www.yachtworld.com/boats/1998/Dufour-43...,1998
2,Greece,,False,65.0000,,,(other),Engine/Fuel Type: Diesel\nEngine Brand: Volvo\...,Fiberglass,46426-2281076,44',Greece,1995 Dromor Athena 44,"EUR 65,000 (US$ 72,560)","US$ 72,560",http://www.yachtworld.com/boats/1995/Dromor-At...,1995
3,Italy,,False,85.0000,,,(other),LOA: 13.30 m\nGenerator\nFresh Water Tanks: (4...,Fiberglass,57770-2854614,44',"Italia, Italy",1999 Cantiere del pardo Grand Soleil 43',"EUR 85,000 (US$ 94,886)","US$ 94,886",http://www.yachtworld.com/boats/1999/Cantiere-...,1999
4,Croatia,Furling mainsail Furling genoa,True,89.0000,Not Paid,Bow thruster,ELAN IMPRESSION,Compass\nLOA: 44 ft 0 in\nFurling mainsail\nFr...,Fiberglass,1590-2971057,44',Croatia,2005 Elan Impression 434,"EUR 89,000 Tax Not Paid (US$ 99,351)","US$ 99,351",http://www.yachtworld.com/boats/2005/Elan-Impr...,2005
5,Greece,,False,69.0000,Not Paid,,BAVARIA CRUISER,The Company offers the details of this vessel ...,Fiberglass,74835-2339541,44',"Athens, Greece",2005 Bavaria 42 Cruiser,"EUR 69,000 Tax Not Paid (US$ 77,025)","US$ 77,025",http://www.yachtworld.com/boats/2005/Bavaria-4...,2005
6,Italy,,False,90.0000,,,JEANNEAU SUN FAST,LOA: 13.30 m\nFresh Water Tanks: (400 Liters)\...,Fiberglass,57770-2936982,44',"Italia, Italy",2003 Jeanneau Sun Fast 43’,"EUR 90,000 (US$ 100,467)","US$ 100,467",http://www.yachtworld.com/boats/2003/Jeanneau-...,2003
7,Croatia,,False,79.0000,Not Paid,,ELAN IMPRESSION,The Company offers the details of this vessel ...,Fiberglass,62889-2484520,44',"At request, Croatia",2006 Elan 434 IMPRESSION,"EUR 79,000 Tax Not Paid (US$ 88,188)","US$ 88,188",http://www.yachtworld.com/boats/2006/Elan-434-...,2006
8,Greece,,False,69.0000,Not Paid,,BAVARIA CRUISER,The Company offers the details of this vessel ...,Fiberglass,74835-2566037,44',"Athens, Greece",2005 Bavaria 42 Cruiser,"EUR 69,500 Tax Not Paid (US$ 77,583)","US$ 77,583",http://www.yachtworld.com/boats/2005/Bavaria-4...,2005
9,Greece,Furling mainsail Furling genoa,True,73.0000,Not Paid,,BAVARIA CRUISER,Furling mainsail\nAutopilot\nBattery charger\n...,Fiberglass,74835-2452699,44',"Kos, Greece",2005 Bavaria 42 Cruiser,"EUR 73,500 Tax Not Paid (US$ 82,048)","US$ 82,048",http://www.yachtworld.com/boats/2005/Bavaria-4...,2005


In [7]:
#yachts['year'].hist(bins=50)

writer = pd.ExcelWriter(OUT_PATH)
yachts.to_excel(writer,'Yachts')
writer.save()